RNN for image classification:

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
#for splitting the data into train/test sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
folder_dir="images"

In [3]:
data=[]
label=[]
SIZE=128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir,folder)):
        if file.endswith("jpg") or file.endswith("png"):
            label.append(folder) 
            #for saving what all classes we are having
            img=cv2.imread(os.path.join(folder_dir,folder,file))
            img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            im=cv2.resize(img_rgb,(SIZE,SIZE))
            data.append(im)
        else:
            continue

In [4]:
data_arr=np.array(data)
label_arr=np.array(label)


In [18]:
encoder=LabelEncoder()
y=encoder.fit_transform(label_arr)
y=to_categorical(y,5) #becaue we have 5 different categories
x=data_arr/255

In [27]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [20]:
# model = tf.keras.Sequential([
#   #tf.keras.Input(shape=(128,128)),
#   keras.layers.Input(input_shape=(128, 128, 3)),
#   tf.keras.layers.GRU(28),
#   tf.keras.layers.Dense(128, activation='relu',input_shape=(128, 128, )),
#   tf.keras.layers.Dropout(0.2,input_shape=(128,)),
#   tf.keras.layers.Dense(10, activation='softmax')
# ])
model = tf.keras.Sequential([
    keras.layers.Input(shape=(128, 128, 3)),  # Input layer for image data
    tf.keras.layers.Reshape((128, 128*3)),   # Reshape to (timesteps, features)
    tf.keras.layers.GRU(128),               # GRU layer
    #tf.keras.layers.Dense(10, activation='softmax'),  # Output layer for 10 classes
    tf.keras.layers.Dense(10, activation='softmax')
])


In [21]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
def scheduler(epoch, lr):
      if epoch < 8:
        return lr
      else:
        return lr * tf.math.exp(-0.1)

In [23]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=2),
    tf.keras.callbacks.LearningRateScheduler(scheduler)
]

In [24]:
print("x_train shape:", x_train.shape)  # Should be (num_samples, height, width, channels)
print("y_train shape:", y_train.shape)  # Should be (num_samples,)

x_train shape: (3108, 128, 128, 3)
y_train shape: (3108, 5)


In [28]:
# Normalize input data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Option 1: Convert one-hot encoded labels to integers
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use categorical_crossentropy if keeping one-hot labels
    metrics=['accuracy']
)

# Train the model
trainer = model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=my_callbacks  # Ensure this is defined, or remove if not used
)


Epoch 1/30
98/98 [==============================] - 15s 129ms/step - loss: 0.7726 - accuracy: 0.7548 - val_loss: 0.8267 - val_accuracy: 0.7301 - lr: 0.0010
Epoch 2/30
98/98 [==============================] - 10s 100ms/step - loss: 0.7555 - accuracy: 0.7535 - val_loss: 0.7945 - val_accuracy: 0.7237 - lr: 0.0010
Epoch 3/30
98/98 [==============================] - 12s 125ms/step - loss: 0.7464 - accuracy: 0.7510 - val_loss: 0.7502 - val_accuracy: 0.7429 - lr: 0.0010
Epoch 4/30
98/98 [==============================] - 16s 159ms/step - loss: 0.7372 - accuracy: 0.7580 - val_loss: 0.8274 - val_accuracy: 0.7211 - lr: 0.0010
Epoch 5/30
98/98 [==============================] - 17s 178ms/step - loss: 0.7247 - accuracy: 0.7580 - val_loss: 0.7359 - val_accuracy: 0.7494 - lr: 0.0010
Epoch 6/30
98/98 [==============================] - 18s 181ms/step - loss: 0.6991 - accuracy: 0.7667 - val_loss: 0.7088 - val_accuracy: 0.7519 - lr: 0.0010
Epoch 7/30
98/98 [==============================] - 17s 178ms/st

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

# Get predictions
y_pred = model.predict(x_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels

# Convert y_test to class labels (if it is one-hot encoded)
if len(y_test.shape) > 1:  # Check if y_test is one-hot encoded
    y_test_classes = y_test.argmax(axis=1)
else:
    y_test_classes = y_test  # Already in integer form

# Calculate precision, recall, and F1 score
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
recall = recall_score(y_test_classes, y_pred_classes, average='macro')
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')

# Print results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


25/25 [==============================] - 4s 74ms/step
Precision: 0.4944
Recall: 0.4424
F1 Score: 0.4452


d:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
